In [3]:
try:
    infile = snakemake.input[0]
    outfile = snakemake.output.csv
    out_html = snakemake.output.html
    templatepath = snakemake.params.template
except:
    infile = "snakedir/SPOG"
    outfile = "snakedir/SPOG.csv"
    out_html = "snakedir/SPOG.html"
    templatepath = "template.html"
import pandas as pd
import lxml.etree as ET
from pathlib import Path


files = list(Path(infile).glob("*.csv"))
df = pd.concat([pd.read_csv(i) for i in files]).fillna("-")
if "iriss" in infile:
    df["speech"] = (
        df.file.str.replace(".xml", "")
        .str.replace("SPOG-", "")
        .str.replace("Iriss-", "")
        .str.strip()
    )
else:
    df["speech"] = (
        df.file.str.replace(".xml", "")
        .str.replace("SPOG-", "")
        .str.replace("Iriss-", "")
        .str.strip()
        .apply(lambda s: s.split("-")[0])
    )
speakers = (
    pd.read_csv("Gos.TEI/Gos-speakers.tsv", sep="\t")
    .fillna("-")
    .astype(str)
    .reset_index(drop=True)
)
speakers = speakers[speakers["PRS-ID"] != "audience"].reset_index(drop=True)
speeches = (
    pd.read_csv("Gos.TEI/Gos-speeches.tsv", sep="\t", index_col=False)
    .fillna("-")
    .reset_index(drop=True)
).astype(str)
speeches["speech"] = speeches["TEXT-ID"].str.replace("Artur-", "").str.strip()

print("Before merging:", df.shape)
df = df.merge(
    speakers["PRS-ID PERM-RESD CHILD-RESD SEX AGE".split()],
    left_on="who",
    right_on="PRS-ID",
    how="left",
)
print("After first merging:", df.shape)
df = df.merge(
    speeches["speech DOMAIN CHANNEL SUBCORPUS TYPE".split()],
    left_on="speech",
    right_on="speech",
    how="left",
)
print("After second merging:", df.shape)
print("After dedup :", df.drop_duplicates().shape)
# c1 = df["PERM-RESD"].str.strip() == "-"
# df.loc[c1, "PERM-RESD"] = df["CHILD-RESD"][c1]
# c2 = df["CHILD-RESD"].str.strip() == "-"
# df.loc[c2, "CHILD-RESD"] = df["PERM-RESD"][c2]
df = df.drop_duplicates().reset_index(drop=True)
# df.to_csv(outfile, index=False)


from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader("."))
template = env.get_template(name=templatepath)
tip_diskurza = (
    df.groupby(["TYPE", "CHANNEL"])
    .agg({"file": "nunique", "numw": "sum"})
    .rename(columns={"numw": "Število besed", "file": "Število posnetkov"})
)
spol = (
    df.groupby("SEX")
    .agg({"who": "nunique", "numw": "sum"})
    .rename(columns={"who": "Število govorcev", "numw": "Število besed"})
)
starost = (
    df.groupby("AGE")
    .agg({"who": "nunique", "numw": "sum"})
    .rename(columns={"who": "Število govorcev", "numw": "Število besed"})
)
statisticnaregija = (
    df.groupby("PERM-RESD")
    .agg({"who": "nunique", "numw": "sum"})
    .rename(columns={"who": "Število govorcev", "numw": "Število besed"})
)

html = template.render(
    subcorpus=str(Path(infile).name),
    tip_diskurza=tip_diskurza,
    spol=spol,
    starost=starost,
    statisticnaregija=statisticnaregija,
)

# Path(out_html).write_text(html)


Before merging: (61, 5)
After first merging: (63, 10)
After second merging: (63, 14)
After dedup : (61, 14)


/tmp/ipykernel_134945/3052960208.py:41: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  pd.read_csv("Gos.TEI/Gos-speeches.tsv", sep="\t", index_col=False)


In [4]:
tip_diskurza

Število posnetkov  Število besed
TYPE        CHANNEL                                      
nezasebni   osebni stik                  5           2347
            telefon                      1            425
razvedrilni radio                        2            874
            televizija                   2            817
zasebni     osebni stik                 10           4301
            telefon                      2            855